In [2]:
function rect_min_max(r)
    x1y1 = r[:,1:2]
    x2y2 = x1y1.+r[:,3:end]
    x1y1,x2y2
end

function box_iou(o,h)
    o1, o2 = rect_min_max(o)
    h1, h2 = rect_min_max(h)
    
    o1 = reshape(o1,(size(o1)[1],1,size(o1)[2]))
    o2 = reshape(o2,(size(o2)[1],1,size(o2)[2]))
    h1 = reshape(h1,(1,size(h1)...))
    h2 = reshape(h2,(1,size(h2)...))
    
    i_min = max.(o1,h1)
    i_max = min.(o2,h2)
    
    i_size = max.(i_max.-i_min, 0)
    i_vol = i_size[:,:,1].*i_size[:,:,2]
    
    # Get volume of union.
    o_size = max.(o2 .- o1, 0)
    h_size = max.(h2 .- h1, 0)
    o_vol = o_size[:,:,1].*o_size[:,:,2]
    h_vol = h_size[:,:,1].*h_size[:,:,2]
    u_vol = o_vol .+ h_vol .- i_vol
    
    i_vol./u_vol
end

function iou_matrix(objs,hyps)
    if size(objs)[1] == 0 || size(hyps)[1] == 0
        return nothing
    end
    box_iou(objs,hyps)
end

keepdim(x) = reshape(x,1,size(x)[1])

keepdim (generic function with 1 method)

In [ ]:
using DelimitedFiles

seqs = ["MOT17-02-DPM", "MOT17-02-SDP", "MOT17-02-FRCNN", "MOT17-04-DPM", "MOT17-04-SDP", "MOT17-04-FRCNN", "MOT17-05-DPM", "MOT17-05-SDP", "MOT17-05-FRCNN", "MOT17-09-DPM", "MOT17-09-SDP", "MOT17-09-FRCNN","MOT17-10-DPM", "MOT17-10-SDP", "MOT17-10-FRCNN", "MOT17-11-DPM", "MOT17-11-SDP", "MOT17-11-FRCNN", "MOT17-13-DPM", "MOT17-13-SDP", "MOT17-13-FRCNN"]
#seq = "MOT17-02-DPM"
for seq in seqs
    dets = readdlm("data/MOT17/train/"*seq*"/det/det.txt", ',', '\n')

    frames = unique(dets[:,1])

    active_tracks = []
    finished_tracks = []

    for frame in frames
        frame_dets = dets[dets[:,1].==frame,:]
        updated_tracks = []

        for track in active_tracks
            if size(frame_dets)[1] > 0
                ious = iou_matrix(keepdim(track[end][3:6]),frame_dets[:,3:6])
                bi = findmax(ious)[2][2]                # index of best frame_det
                push!(track,frame_dets[bi,:])           # assign det to track
                push!(updated_tracks, track)            # so we know which tracks were updated now
                frame_dets = frame_dets[1:end .!= bi,:] # delete the det
            end

        end

        # create new tracks
        if length(updated_tracks) != 0
            for updated_track in updated_tracks
                push!(active_tracks,updated_track)
            end
        end
        if size(frame_dets)[1] != 0
            for new_track in eachrow(frame_dets)
                push!(active_tracks,push!([],new_track[:]))
            end
        end
    end

    for active_track in active_tracks
        push!(finished_tracks, active_track)
    end

    # write IDs
    for ti in 1:size(finished_tracks,1)
        for di in 1:size(finished_tracks[ti],1)
            finished_tracks[ti][di][2] = ti
        end
    end
    
    # save results
    open(string("results/",seq,".txt"), "w") do io
        writedlm(io, vcat(finished_tracks),'\n')
    end
end

In [23]:
seq+"txt"

LoadError: MethodError: no method matching +(::String, ::String)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560

In [16]:
import Pkg
Pkg.add("Conda")
using Conda

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
    Updating `/scratch/users/tanjary21/.julia/environments/v1.6/Project.toml`
  [8f4d0f93] + Conda v1.5.2
  No Changes to `/scratch/users/tanjary21/.julia/environments/v1.6/Manifest.toml`


In [20]:
Pkg.add("PyCall")

   Resolving package versions...
   Installed MacroTools ─ v0.5.9
   Installed PyCall ───── v1.92.5
    Updating `/scratch/users/tanjary21/.julia/environments/v1.6/Project.toml`
  [438e738f] + PyCall v1.92.5
    Updating `/scratch/users/tanjary21/.julia/environments/v1.6/Manifest.toml`
  [1914dd2f] + MacroTools v0.5.9
  [438e738f] + PyCall v1.92.5
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/4ba3651d33ef76e24fef6a598b63ffd1c5e1cd17/build.log`
Precompiling project...
  ✓ MacroTools
  ✓ PyCall
  2 dependencies successfully precompiled in 12 seconds (16 already precompiled)


In [28]:
Conda.add_channel("loopbio")
Conda.add("motmetrics")

┌ Info: Running `conda config --add channels loopbio --file /kuacc/users/tanjary21/.julia/conda/3/condarc-julia.yml --force` in root environment
└ @ Conda /kuacc/users/tanjary21/.julia/packages/Conda/sNGum/src/Conda.jl:128
┌ Info: Running `conda install -y motmetrics` in root environment
└ @ Conda /kuacc/users/tanjary21/.julia/packages/Conda/sNGum/src/Conda.jl:128


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 

Examining conflict for python motmetrics:   0%|          | 0/7 [00:00<?, ?it/s]


Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed


                                                                               
UnsatisfiableError: The following specifications were found to be incompatible with each other:

Output in format: Requested package -> Available versionsThe following specifications were found to be incompatible with your system:

  - feature:/linux-64::__glibc==2.17=0
  - feature:|@/linux-64::__glibc==2.17=0

Your installed version is: 2.17




LoadError: failed process: Process(setenv(`[4m/kuacc/users/tanjary21/.julia/conda/3/bin/conda[24m [4minstall[24m [4m-y[24m [4mmotmetrics[24m`,["PATH=/kuacc/apps/anaconda/3.6/bin:/kuacc/apps/anaconda/3.6:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/kuacc/apps/scripts:/kuacc/users/tanjary21/.local/bin:/kuacc/users/tanjary21/bin", "SLURM_TASKS_PER_NODE=4", "SLURM_NODEID=0", "SLURM_JOB_GID=200513", "SLURM_JOB_NAME=J_J", "QTDIR=/usr/lib64/qt-3.3", "SLURM_JOBID=2541977", "MODULEPATH=/kuacc/etc/mod", "SLURM_MEM_PER_NODE=20480", "USER=tanjary21"  …  "MAIL=/var/spool/mail/tanjary21", "CONDA_PREFIX=/kuacc/users/tanjary21/.julia/conda/3", "LC_NAME=tr_TR.UTF-8", "XDG_SESSION_ID=40578", "KDEDIRS=/usr", "LC_IDENTIFICATION=tr_TR.UTF-8", "SLURM_PROCID=0", "SLURM_NNODES=1", "LS_COLORS=rs=0:di=38;5;27:ln=38;5;51:mh=44;38;5;15:pi=40;38;5;11:so=38;5;13:do=38;5;5:bd=48;5;232;38;5;11:cd=48;5;232;38;5;3:or=48;5;232;38;5;9:mi=05;48;5;232;38;5;15:su=48;5;196;38;5;15:sg=48;5;